In [250]:
using Random, Distributions, Statistics

In [33]:
large_class = Dict(:flat => (46,9,1),:med => (40,10,3), :gen => (34,11,5))
small_class = Dict(:flat => (23,4,1),:med => (19,5,2), :gen => (15,6,3))

Dict{Symbol,Tuple{Int64,Int64,Int64}} with 3 entries:
  :flat => (23, 4, 1)
  :gen  => (15, 6, 3)
  :med  => (19, 5, 2)

In [41]:
sa = Dict()
sa[:A]=(:L1,:L6,:L12,:S1)
sa[:B]=(:L1,:L7,:L13,:S2)
sa[:C]=(:L2,:L7,:L9,:S3)
sa[:D]=(:L3,:L8,:L10,:S4)
sa[:E]=(:L4,:L8,:L12,:S5)
sa[:F]=(:L2,:L10,:L14,:S6)
sa[:G]=(:L4,:L6,:L14,:S7)
sa[:H]=(:L5,:L11,:L13,:S8)
sa[:I]=(:L3,:L11,:L15,:S9)
sa[:J]=(:L5,:L9,:L15,:S10)

(:L5, :L9, :L15, :S10)

In [51]:
class_type = Dict()
for i=1:5
    class_type[Symbol("L",i)] = :crim
end

for i=6:8
    class_type[Symbol("L",i)] = :civ_pro
end

for i=9:11
    class_type[Symbol("L",i)] = :contracts
end

for i=12:15
    class_type[Symbol("L",i)] = :torts
end

for i in [1,2,5,7]
    class_type[Symbol("S",i)] = :contracts
end

for i in [3,4]
    class_type[Symbol("S",i)] = :torts
end

for i in [6,8,9,10]
    class_type[Symbol("S",i)] = :civ_pro
end

In [400]:
classes = foldl(vcat,map(collect,values(sa) |> collect)) |> union
sections = keys(sa) |> collect |> sort

10-element Array{Any,1}:
 :A
 :B
 :C
 :D
 :E
 :F
 :G
 :H
 :I
 :J

In [401]:
function get_class_grades(classes::Array, tup)
    @assert tup[1] |> isodd
    @assert tup[2] |> iseven
    
    l_classes = filter(x->string(x)[1]=='L', classes) |> shuffle
    @assert length(l_classes) - 2 >= tup[1]
    
    s_l = (length(l_classes) - tup[1])/2 |> x->(convert(Int,x))
    
    s_classes = filter(x->string(x)[1]=='S', classes) |> shuffle
    @assert length(l_classes) - 2 >= tup[2]
    
    s_s = (length(s_classes) - tup[2])/2 |> x->(convert(Int,x))


    class_grades = Dict()
    
    j = 1
    for i=1:tup[1]
        class_grades[l_classes[i]] = generate_sample(large_class[:med])
        j += 1
    end
    
    k = j
    for i=j:(j + s_l -1)
        class_grades[l_classes[i]] = generate_sample(large_class[:flat])
        k+=1
    end
    
    for i=k:(k + s_l -1)
        class_grades[l_classes[i]] = generate_sample(large_class[:gen])
    end
    
    # small section classes
    j = 1
    for i=1:tup[2]
        class_grades[s_classes[i]] = generate_sample(small_class[:med])
        j += 1
    end
    
    k = j
    for i=j:(j + s_s - 1)
        class_grades[s_classes[i]] = generate_sample(small_class[:flat])
        k +=1
    end
    
    for i=k:(k + s_s - 1)
        class_grades[s_classes[i]] = generate_sample(small_class[:gen])
    end
    
    
    class_grades
end

get_class_grades (generic function with 2 methods)

In [402]:
students = Dict()
all_students = []
for s in keys(sa)
    arr = Vector{Symbol}()
    for i=1:33
        si = Symbol(string(s),i)
        push!(arr,si)
        push!(all_students, si)
    end
    students[s] = arr
end
student_grades = Dict()
for s in all_students
    student_grades[s] = []
end

In [403]:
function generate_sample(tup, shuf = true)
    o1 = ones(tup[1]) * 3.3
    o2 = ones(tup[2]) * 3.7
    o3 = ones(tup[2]) * 3.0
    o4 = ones(tup[3]) * 4.0
    o5 = ones(tup[3]) * 2.7
    
    grades = foldl(vcat,[o1,o2,o3,o4,o5])
    
    out = shuf ? shuffle(grades) : sort(grades)
end
        

generate_sample (generic function with 2 methods)

In [404]:
shuffle_factor = .05

0.05

In [405]:
mm = 1
dd = .1
dist = Normal(mm,dd)

Normal{Float64}(μ=1.0, σ=0.1)

In [406]:
student_smart = Dict()
for s in all_students
    student_smart[s] = rand(dist)
end

In [407]:
ccc = Dict()
for cl in classes
    ss = []
    for s in keys(sa) 
        if length(intersect(sa[s],[cl])) > 0
            push!(ss, s)
        end
    end
    ccc[cl] = ss
end
ccc

Dict{Any,Any} with 25 entries:
  :L10 => Any[:F, :D]
  :S9  => Any[:I]
  :L11 => Any[:I, :H]
  :L2  => Any[:F, :C]
  :L12 => Any[:A, :E]
  :S1  => Any[:A]
  :S8  => Any[:H]
  :L1  => Any[:A, :B]
  :S4  => Any[:D]
  :L4  => Any[:G, :E]
  :L5  => Any[:H, :J]
  :L14 => Any[:F, :G]
  :L8  => Any[:D, :E]
  :S3  => Any[:C]
  :L13 => Any[:H, :B]
  :S2  => Any[:B]
  :S5  => Any[:E]
  :S10 => Any[:J]
  :L15 => Any[:I, :J]
  :S6  => Any[:F]
  :L6  => Any[:A, :G]
  :S7  => Any[:G]
  :L9  => Any[:J, :C]
  :L3  => Any[:I, :D]
  :L7  => Any[:B, :C]

In [408]:
function get_class_curves(classes::Array, tt_l,tt_s)
    class_curves = Dict()
    
    
    @assert tt_l |> isodd
    @assert tt_s |> iseven

    l_classes = filter(x->string(x)[1]=='L', classes) |> shuffle
    @assert length(l_classes) - 2 >= tt_l

    s_l = (length(l_classes) - tt_l)/2 |> x->(convert(Int,x))

    s_classes = filter(x->string(x)[1]=='S', classes) |> shuffle
    @assert length(l_classes) - 2 >= tt_s

    s_s = (length(s_classes) - tt_s)/2 |> x->(convert(Int,x))

    j = 1
    for i=1:tt_l
        class_curves[l_classes[i]] = :med
        j += 1
    end
    
    k = j
    for i=j:(j+ s_l - 1)
        class_curves[l_classes[i]] = :flat
        k += 1
    end
    
    for i=k:(k + s_l -1)
        class_curves[l_classes[i]] = :gen
    end
    
    j = 1
    for i=1:tt_s
        class_curves[s_classes[i]] = :med
        j += 1
    end
    
    k = j
    for i=j:(j+ s_s - 1)
        class_curves[s_classes[i]] = :flat
        k += 1
    end
    
    for i=k:(k + s_s -1)
        class_curves[s_classes[i]] = :gen
    end
    
    
    class_curves
end

get_class_curves (generic function with 1 method)

In [409]:
class_curves = get_class_curves(classes,11,8)

Dict{Any,Any} with 25 entries:
  :L10 => :med
  :S9  => :med
  :L2  => :med
  :L11 => :med
  :L12 => :gen
  :S8  => :med
  :S1  => :med
  :L1  => :med
  :S4  => :med
  :L4  => :gen
  :L5  => :med
  :L14 => :flat
  :L8  => :med
  :S3  => :med
  :L13 => :med
  :S2  => :med
  :S5  => :gen
  :S10 => :flat
  :L15 => :flat
  :S6  => :med
  :L6  => :med
  :S7  => :med
  :L9  => :med
  :L3  => :med
  :L7  => :med

In [410]:
out_test = []
for cl in classes
    mult = class_type[cl] == :crim ? 3 : 4
    
    st = foldl(vcat,map(x->students[x], ccc[cl]))
    
    s_mod = []
    
    for s in st
        s_factor = (rand() - .5)*(2*shuffle_factor)
        push!(s_mod,(s,student_smart[s] + s_factor))
    end
    
    s_shuffled = sort(s_mod, by = x->x[2])
    this_curve = string(cl)[1] == 'L' ? large_class[class_curves[cl]] : small_class[class_curves[cl]]
    class_grades = generate_sample(this_curve, false) * mult
    
    out1 = []
    
    for (i,x) in enumerate(class_grades)
        push!(out1, (s_shuffled[i][1], x))
    end
    
    push!(out_test, out1)
end    

In [411]:
all_gp = foldl(vcat,out_test)

1320-element Array{Any,1}:
 (:D18, 8.100000000000001)
 (:I11, 8.100000000000001)
 (:D21, 8.100000000000001)
 (:I9, 9.0)               
 (:I3, 9.0)               
 (:D31, 9.0)              
 (:I29, 9.0)              
 (:I31, 9.0)              
 (:I26, 9.0)              
 (:D26, 9.0)              
 (:I4, 9.0)               
 (:D19, 9.0)              
 (:D32, 9.0)              
 ⋮                        
 (:C26, 13.2)             
 (:C23, 13.2)             
 (:C21, 13.2)             
 (:C18, 13.2)             
 (:C32, 13.2)             
 (:C1, 14.8)              
 (:C12, 14.8)             
 (:C2, 14.8)              
 (:C6, 14.8)              
 (:C20, 14.8)             
 (:C24, 16.0)             
 (:C30, 16.0)             

In [412]:
for p in all_gp
    push!(student_grades[p[1]],p[2])
end

In [413]:
student_grades[:E14]

4-element Array{Any,1}:
 13.2              
  9.899999999999999
 13.2              
 14.8              

In [414]:
student_gpas = Dict()
for k in keys(student_grades)
    student_gpas[k] = sum(student_grades[k])/15
end

In [415]:
all_gpas = values(student_gpas)|> collect |> sort

330-element Array{Any,1}:
 2.7               
 2.7               
 2.7               
 2.7               
 2.7               
 2.7800000000000002
 2.7800000000000002
 2.7800000000000002
 2.7800000000000002
 2.8400000000000003
 2.8400000000000003
 2.8400000000000003
 2.8400000000000003
 ⋮                 
 3.8600000000000003
 3.92              
 3.92              
 3.92              
 3.92              
 3.92              
 3.92              
 3.94              
 4.0               
 4.0               
 4.0               
 4.0               

In [416]:
for i =.05:.05:.95
    println("the ",i," percentile is: ", quantile(all_gpas, i))
end

the 0.05 percentile is: 2.92
the 0.1 percentile is: 3.0
the 0.15 percentile is: 3.08
the 0.2 percentile is: 3.1600000000000006
the 0.25 percentile is: 3.2199999999999998
the 0.3 percentile is: 3.2399999999999998
the 0.35 percentile is: 3.3
the 0.4 percentile is: 3.3
the 0.45 percentile is: 3.3
the 0.5 percentile is: 3.3
the 0.55 percentile is: 3.3
the 0.6 percentile is: 3.3
the 0.65 percentile is: 3.3
the 0.7 percentile is: 3.3
the 0.75 percentile is: 3.4066666666666663
the 0.8 percentile is: 3.4866666666666664
the 0.85 percentile is: 3.610666666666666
the 0.9 percentile is: 3.7
the 0.95 percentile is: 3.8130000000000006


In [224]:
# no intelligence; random grades
for cl in keys(all_grades)
    mult = class_type[cl] == :crim ? 3 : 4
    
    st = foldl(vcat,map(x->students[x], ccc[cl]))
    gr = all_grades[cl]
    @assert length(st) == length(gr)
    
    for (i,s) in enumerate(st)
        push!(student_grades[s],gr[i] * mult)
    end
end

In [225]:
student_grades[:A11]

4-element Array{Any,1}:
 13.2              
 13.2              
  9.899999999999999
 10.8              

In [226]:
gpas = []
for arr in values(student_grades)
    push!(gpas, sum(arr)/15)
end

In [227]:
using Statistics

In [232]:
quantile(gpas,.997)

3.733939999999999

In [229]:
median(gpas)

3.3